0. Create a local workspace
1. USE gpt-4.0 as the model.
2. Find a good dataset to work on.
3. Try various PEFT techniques:
    - LORA
    - QLORA (bitsandbytes)
    - etc. (https://pypi.org/project/peft/0.7.0/)

4. Create a GitHub WebPage